## 104 人力銀行資料分析師職缺的工作描述

郭耀仁

## 載入筆記本需要的套件

In [1]:
import requests
import time
import random
import re
import pandas as pd

## 搜尋「資料分析師」

- 全職
- 經歷要求：1 年以下

![Imgur](https://i.imgur.com/C8Sf3D6.png?1)

## 以 Chrome 開發者工具檢視 HTTP 請求/回應

- 請求種類：`GET`
- 請求 URL：https://www.104.com.tw/jobs/search/list

![Imgur](https://i.imgur.com/WP2mHg9.png?1)

In [2]:
request_url = "https://www.104.com.tw/jobs/search/list"

- Request Headers

In [3]:
request_headers = {
    'Referer': 'https://www.104.com.tw/jobs/search/'
}

- Query String Parameters

In [4]:
query_string_parameters = {
    'ro': '1',
    'kwop': '7',
    'keyword': '資料分析師',
    'jobcatExpansionType': '0',
    'order': '15',
    'asc': '0',
    'page': '1',
    'jobexp': '1',
    'mode': 's',
    'jobsource': '2018indexpoc'
}

## 發送請求

In [5]:
response = requests.get(request_url, params=query_string_parameters, headers=request_headers)
print(response.status_code)

200


## 解析回應

In [6]:
response_json = response.json()
n_jobs = response_json['data']['totalCount']
n_pages = response_json['data']['totalPage']
print("符合搜尋條件的職缺有{}筆，分佈在{}頁".format(n_jobs, n_pages))

符合搜尋條件的職缺有113筆，分佈在6頁


## 「有禮貌」地爬取職缺的頁面連結

- 一次爬取一頁
- 間隔 5-10 秒再爬取下一頁

In [7]:
def get_job_links():
    job_links = []
    for page in range(1, n_pages + 1):
        query_string_parameters = {
            'ro': '1',
            'kwop': '7',
            'keyword': '資料分析師',
            'jobcatExpansionType': '0',
            'order': '15',
            'asc': '0',
            'page': str(page),
            'jobexp': '1',
            'mode': 's',
            'jobsource': '2018indexpoc'
        }
        response = requests.get(request_url, params=query_string_parameters, headers=request_headers)
        response_json = response.json()
        for job in response_json['data']['list']:
            job_link = job['link']['job'].replace('//', '')
            job_link = "https://{}".format(job_link)
            job_links.append(job_link)
        print("正在爬取第{}頁的職缺...".format(page))
        sleep_time = random.randint(5, 10)
        time.sleep(sleep_time)
    return job_links

In [8]:
job_links = get_job_links()

正在爬取第1頁的職缺...
正在爬取第2頁的職缺...
正在爬取第3頁的職缺...
正在爬取第4頁的職缺...
正在爬取第5頁的職缺...
正在爬取第6頁的職缺...


In [9]:
len(job_links)

113

## 以 Chrome 開發者工具檢視 HTTP 請求/回應

- 請求種類：`GET`
- 請求 URL：https://www.104.com.tw/job/ajax/content/{job_id}

![Imgur](https://i.imgur.com/dq2TaNE.png)

## 「有禮貌」地爬取職缺的工作描述

- 間隔 1-5 秒再爬取下一個職缺的工作描述

In [10]:
def get_job_desc(job_links):
    job_ids = [re.split("/|\?", i)[-2] for i in job_links]
    job_descriptions = []
    for job_id, job_link in zip(job_ids, job_links):
        request_url = "https://www.104.com.tw/job/ajax/content/{}".format(job_id)
        request_header = {
            'Referer': job_link
        }
        response = requests.get(request_url, headers=request_headers)
        response_json = response.json()
        company = response_json['data']['header']['custName']
        job_title = response_json['data']['header']['jobName']
        job_desc = response_json['data']['jobDetail']['jobDescription']
        job_desc = re.sub("\n|\r", "", job_desc)
        job_data = {
            'company': company,
            'jobTitle': job_title,
            'jobDesc': job_desc
        }
        job_descriptions.append(job_data)
        print("正在爬取{}的{}工作描述...".format(company, job_title))
        sleep_time = random.randint(1, 5)
        time.sleep(sleep_time)
    return job_descriptions

In [11]:
job_desc = get_job_desc(job_links)

正在爬取17直播_英屬維京群島商麻吉一七股份有限公司臺灣分公司的(Jr./Sr.) Data Analyst 資深資料分析師工作描述...
正在爬取烏龜移動科技股份有限公司的資料分析師工作描述...
正在爬取PIXNET_優像數位媒體科技股份有限公司的資料分析師(NC-痞客邦研發中心)工作描述...
正在爬取核桃運算股份有限公司的Data Analyst 資料分析師工作描述...
正在爬取永慶房產集團_永慶房屋仲介股份有限公司的資料分析師.工作描述...
正在爬取艾思網絡股份有限公司的商業資料分析師 Business Data Analyst工作描述...
正在爬取瑞嘉軟體科技股份有限公司的Data Analyst / 資料分析師工作描述...
正在爬取童綜合醫療社團法人童綜合醫院的童綜合醫院資料分析師工作描述...
正在爬取意藍資訊股份有限公司的資料分析師 (社群資料、人群資料)工作描述...
正在爬取薩摩亞商俠客行軟件科技有限公司的資料分析師(台北上班)工作描述...
正在爬取薩摩亞商俠客行軟件科技有限公司的資料分析師(派駐菲律賓)工作描述...
正在爬取聯強國際股份有限公司的資料分析師(全球第二大IT通路集團)工作描述...
正在爬取天氣風險管理開發股份有限公司的資料分析師工作描述...
正在爬取新代科技股份有限公司的資料數據分析師工作描述...
正在爬取優愛德股份有限公司的資料分析師 Data Analyst工作描述...
正在爬取永豐金證券股份有限公司的資料分析師(資訊部)工作描述...
正在爬取明基電通股份有限公司的資料分析師【明基BenQ雲端創新中心】工作描述...
正在爬取凱鈿行動科技股份有限公司的資料分析師 Data Analyst (台南/Tainan)工作描述...
正在爬取意藍資訊股份有限公司的產品分析師 (PeopleView人群資料庫)工作描述...
正在爬取OK超商_來來超商股份有限公司的【總公司】 資料分析專員(資訊部)-A01工作描述...
正在爬取中國醫藥大學附設醫院的1M94-醫療智慧中心臨床資料組 系統分析師工作描述...
正在爬取中國醫藥大學附設醫院的1M94-醫療智慧中心臨床資料組 初級臨床數據分析師工作描述...
正在爬取國泰金控_國泰金融控股股份有限公司的資料科學分析師（Data Science R&D）- 資料科學實

## （Optional）將爬取好的資料輸出成試算表

In [12]:
df = pd.DataFrame(job_desc)
df.head()

,company,jobTitle,jobDesc
0,17直播_英屬維京群島商麻吉一七股份有限公司臺灣分公司,(Jr./Sr.) Data Analyst 資深資料分析師,17 Media is the world leader in the live strea...
1,烏龜移動科技股份有限公司,資料分析師,蝙蝠移動的核心服務是提供客戶專屬的私有化 DMP（Data Management Platf...
2,PIXNET_優像數位媒體科技股份有限公司,資料分析師(NC-痞客邦研發中心),作為大數據資料分析師，工作內容將會協同業務、廣告、商務、會員與內容等經營單位，從經營角度協助...
3,核桃運算股份有限公司,Data Analyst 資料分析師,此職務為執行數據服務專案的角色，需具備高度的數據素養，以及相應技術與知識；如果你/妳富有責任...
4,永慶房產集團_永慶房屋仲介股份有限公司,資料分析師.,永慶房產集團垂直整合三大事業群，10年成長25倍，歡迎具備邏輯思考、分析判斷及溝通能力的優秀...


In [13]:
df.to_excel('da_job_desc_104.xlsx', index=False)